In [1]:
import os
import random
import pathlib
import tensorflow as tf
import tensorflow.io as tfio
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models

In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
NUM_CLASSES = 2
RESIZE_TO = 512
CROP_TO = 256
IMAGE_SIZE = (CROP_TO, CROP_TO)
NUM_CHANNELS = 3
SCHEDULE_LENGTH = 1000
SCHEDULE_BOUNDARIES = [300, 600, 900]
BATCH_SIZE = 64
SCHEDULE_LENGTH = SCHEDULE_LENGTH * 512 / BATCH_SIZE
STEPS_PER_EPOCH = 10
lr = 0.03 * BATCH_SIZE / 512 

def load_and_preprocess_image(path, target_image_shape=IMAGE_SIZE, num_channnels=NUM_CHANNELS):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=num_channnels)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, 0.2, seed=None)
    image = tf.image.random_hue(image, 0.2)
#     image = tf.image.resize(image, [RESIZE_TO, RESIZE_TO])
#     image = tf.image.random_crop(image, [CROP_TO, CROP_TO, 3])
    image = tf.image.resize(image, [CROP_TO, CROP_TO])
    image /= 255.0  # normalize to [0,1] range
#     image = 2*image-1
    return image

In [3]:
train_val_data_root = '../dataset_5_512/train'
train_val_data_root = pathlib.Path(train_val_data_root)
all_image_paths = [str(path) for path in list(train_val_data_root.glob('*/*'))]
label_names = sorted(item.name for item in train_val_data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index,name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
image_count = len(all_image_paths)
image_path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = image_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
train_pipeline = (
    image_label_ds
        .cache()
        .apply(tf.data.experimental.shuffle_and_repeat(buffer_size=len(all_image_paths)))
        .batch(BATCH_SIZE)
        .prefetch(buffer_size=AUTOTUNE)
)
train_steps_per_epoch=tf.math.ceil(len(all_image_paths)/BATCH_SIZE).numpy()

Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.


In [4]:
test_data_root = '../dataset_5_512/val'
test_data_root = pathlib.Path(test_data_root)
all_image_paths = [str(path) for path in list(test_data_root.glob('*/*'))]
label_names = sorted(item.name for item in test_data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index,name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
image_count = len(all_image_paths)
image_path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = image_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
test_pipeline = (image_label_ds
                    .cache()
                    .apply(tf.data.experimental.shuffle_and_repeat(buffer_size=len(all_image_paths)))
                    .batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE))
val_steps_per_epoch=tf.math.ceil(len(all_image_paths)/BATCH_SIZE).numpy()

In [8]:
input_tensor = tf.keras.Input(shape=(CROP_TO,CROP_TO,NUM_CHANNELS))
vgg16 =  tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
x = vgg16.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=vgg16.input, outputs=x)

In [9]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
)
loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(
    loss=loss_fn,
    optimizer=optimizer,
    metrics=['accuracy']
)

In [10]:
history = model.fit(
    train_pipeline, 
    epochs=100, 
    steps_per_epoch=train_steps_per_epoch, 
    validation_data=test_pipeline, 
    validation_steps=val_steps_per_epoch
)

Epoch 1/100
498/498 [==============================] - 222s 446ms/step - loss: 0.1851 - accuracy: 0.9223 - val_loss: 0.1269 - val_accuracy: 0.9500
Epoch 2/100
498/498 [==============================] - 225s 452ms/step - loss: 0.0915 - accuracy: 0.9652 - val_loss: 0.1221 - val_accuracy: 0.9539
Epoch 3/100
498/498 [==============================] - 225s 452ms/step - loss: 0.0714 - accuracy: 0.9730 - val_loss: 0.1194 - val_accuracy: 0.9566
Epoch 4/100
498/498 [==============================] - 224s 450ms/step - loss: 0.0523 - accuracy: 0.9806 - val_loss: 0.2177 - val_accuracy: 0.9311
Epoch 5/100
498/498 [==============================] - 223s 448ms/step - loss: 0.0418 - accuracy: 0.9847 - val_loss: 0.0934 - val_accuracy: 0.9653
Epoch 6/100
498/498 [==============================] - 223s 448ms/step - loss: 0.0304 - accuracy: 0.9884 - val_loss: 0.1663 - val_accuracy: 0.9504
Epoch 7/100
498/498 [==============================] - 223s 448ms/step - loss: 0.0245 - accuracy: 0.9907 - val_loss: 0

Epoch 56/100
498/498 [==============================] - 222s 445ms/step - loss: 1.6386e-06 - accuracy: 1.0000 - val_loss: 0.3239 - val_accuracy: 0.9632
Epoch 57/100
498/498 [==============================] - 222s 446ms/step - loss: 0.0159 - accuracy: 0.9948 - val_loss: 0.1852 - val_accuracy: 0.9630
Epoch 58/100
498/498 [==============================] - 222s 445ms/step - loss: 0.0045 - accuracy: 0.9987 - val_loss: 0.2072 - val_accuracy: 0.9637
Epoch 59/100
498/498 [==============================] - 222s 446ms/step - loss: 0.0020 - accuracy: 0.9994 - val_loss: 0.2154 - val_accuracy: 0.9647
Epoch 60/100
498/498 [==============================] - 222s 445ms/step - loss: 0.0053 - accuracy: 0.9983 - val_loss: 0.1852 - val_accuracy: 0.9628
Epoch 61/100
498/498 [==============================] - 222s 446ms/step - loss: 2.9509e-04 - accuracy: 0.9999 - val_loss: 0.2144 - val_accuracy: 0.9647
Epoch 62/100
498/498 [==============================] - 222s 446ms/step - loss: 3.3025e-05 - accuracy: 1